In [1]:
# env:freecad_fem_env_meshlab 
import os
os.environ["PYOPENGL_PLATFORM"] = "egl"
import pyrender
import trimesh
import numpy as np
import datetime as dt 
from PIL import Image

In [2]:
    
def make_observation(trmsh):
    camera_pres = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
    s = np.sqrt(2)/2
    camera_pres_pose_top = np.array([
    [s, 0.0, s, 5.0],
    [0.0, 1.0, 0.0, 2.0],
    [s, 0.0, s, 10],
    [0.0, 0.0, 0.0, 1.0],
    ])
            
    mish = pyrender.Mesh.from_trimesh(trmsh)
    prscene = pyrender.Scene(ambient_light=np.array([1.0, 1.0, 1.0, 1.0]))
    prscene.add(camera_pres,pose=camera_pres_pose_top)
    prscene.add(mish)
    
    r = pyrender.OffscreenRenderer(viewport_width=640,viewport_height=480,point_size=1.0)
    img,dpth = r.render(prscene)
    # print(f"============={img.size}")
    return img, dpth
    
def trimesh_to_color_depth(trmsh):
    kaf = np.array(trmsh.vertices)
    kaf[:,2]=0

    radii = np.linalg.norm(trmsh.vertices - kaf, axis=1)
    cmap = ['jet' ,'viridis']
    trmsh.visual.vertex_colors = trimesh.visual.interpolate(radii, color_map=cmap[0])
    return trmsh

In [3]:
# when we have files in root and the subfolders
#-----------------------------------------------
#
# for path, subdirs, files in os.walk(root_folder): # load meshes from all subfolders
#     for name in files:
#         fullpath_msh_lst.append(os.path.join(path, name))


# for mytrmesh_fullpath in fullpath_msh_lst:        
#     if not mytrmesh_fullpath.startswith('.') and mytrmesh_fullpath.endswith('.obj'):
#         mytrmesh = trimesh.load(mytrmesh_fullpath)

#         colored_result_mesh = trimesh_to_color_depth(mytrmesh)
#         rgbimg, depth_img = make_observation(colored_result_mesh)
#         im_observation = Image.fromarray(rgbimg).resize([64,48]).crop(box=[5,15,60,38])
    
#         image_filename = mytrmesh_fullpath.replace('obj_Meshes','png_Meshes').replace('.obj', '.png')    
#         save_obs=True
#         if(save_obs):
#             im_observation.save(image_filename)
#             # self.im_observation = np.asarray(self.im_observation) # img or array



In [4]:
# create empty folders for png files - if does not exist

fullpath_msh_lst = []
root_folder = '/work/aifa/DATA/RLFEM_Data/obj_Meshes'

# for path, subdirs, files in os.walk(root_folder): # load meshes from all subfolders
#     for name in subdirs:
#         if os.path.isdir(os.path.join(path,name)):
#             new_folder = os.path.join(path,name).replace('obj_Meshes', 'png_Meshes')
#             if not os.path.exists(new_folder):
#                os.mkdir(new_folder) 

In [5]:
# only search the subfolders, and generate png from trimesh files 
#-----------------------------------------------
sub_f_list = []
for path, subdirs, files in os.walk(root_folder): # load meshes from all subfolders
    for name in subdirs:
        if os.path.isdir(os.path.join(path,name)):
            sub_f_list.append(os.path.join(path,name))
            
sub_f_list.sort()            
            
for sub_f in sub_f_list:
    print(sub_f)
    obj_list = os.listdir(sub_f)
    for mytrmesh in obj_list:
        if not mytrmesh.startswith('.') and mytrmesh.endswith('.obj'):
            mytrmesh_fullpath = os.path.join(sub_f,mytrmesh)
            print(mytrmesh_fullpath)
            mytrmesh = trimesh.load(mytrmesh_fullpath)

            colored_result_mesh = trimesh_to_color_depth(mytrmesh)
            rgbimg, depth_img = make_observation(colored_result_mesh)
            im_observation = Image.fromarray(rgbimg).resize([64,48]).crop(box=[5,15,60,38])
        
            image_filename = mytrmesh_fullpath.replace('obj_Meshes','png_Meshes').replace('.obj', '.png')    
            save_obs=True
            if(save_obs):
                im_observation.save(image_filename)
                